In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld

URM_all, ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_event_all = ld.getCOOs()
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13642 ( 0.1%) Users that have less than 1 test interactions


In [4]:
import os

output_folder_path = "../result_experiments/8_FM"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 50  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [5]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "epochs": Categorical([300]),
    "n_components": Integer(1, 200),
    "loss": Categorical(['bpr', 'warp', 'warp-kos']),
    "sgd_mode": Categorical(['adagrad', 'adadelta']),
    "learning_rate": Real(low = 1e-6, high = 1e-1, prior = 'log-uniform'),
    "item_alpha": Real(low = 1e-5, high = 1e-2, prior = 'log-uniform'),
    "user_alpha": Real(low = 1e-5, high = 1e-2, prior = 'log-uniform'),
}

In [6]:
earlystopping_keywargs = {"validation_every_n": 5,
                          "stop_on_validation": True,
                          "evaluator_object": evaluator_validation,
                          "lower_validations_allowed": 5,
                          "validation_metric": metric_to_optimize,
                          }

In [7]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = earlystopping_keywargs     # Additiona hyperparameters for the fit function
)

In [10]:
from Recommenders.FactorizationMachines.LightFMRecommender import LightFMCFRecommender
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = LightFMCFRecommender

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation)

In [ ]:
hyperparameterSearch.search(recommender_input_args,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

SearchBayesianSkopt: argument save_model is 'last' but no recommender_input_args_last_test provided, saving best model on train data alone.
Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'epochs': 500, 'num_factors': 128, 'sgd_mode': 'adam', 'batch_size': 32, 'item_reg': 1.0319869779160993e-05, 'user_reg': 2.5029350478602385e-05, 'learning_rate': 0.01249084713085261}
MatrixFactorization_FunkSVD_Cython_Recommender: URM Detected 1 ( 0.0%) items with no interactions.
FUNK_SVD: Processed 4228544 (100.0%) in 1.38 min. MSE loss 7.45E-01. Sample per second: 50996
FUNK_SVD: Epoch 1 of 500. Elapsed time 1.37 min
FUNK_SVD: Processed 4228544 (100.0%) in 1.39 min. MSE loss 9.86E-02. Sample per second: 50705
FUNK_SVD: Epoch 2 of 500. Elapsed time 2.74 min
FUNK_SVD: Processed 4228544 (100.0%) in 1.37 min. MSE loss 3.14E-02. Sample per second: 51287
FUNK_SVD: Epoch 3 of 500. Elapsed time 4.11 min
FUNK_SVD: Processed 4228544 (100.0%) in 1.37 min. MS

FUNK_SVD: Processed 4228544 (100.0%) in 1.38 min. MSE loss 1.84E-01. Sample per second: 51050
FUNK_SVD: Epoch 26 of 500. Elapsed time 36.38 min
FUNK_SVD: Processed 4228544 (100.0%) in 1.38 min. MSE loss 8.93E-02. Sample per second: 51099
FUNK_SVD: Epoch 27 of 500. Elapsed time 37.75 min
FUNK_SVD: Processed 4228544 (100.0%) in 1.38 min. MSE loss 1.34E-02. Sample per second: 51173
FUNK_SVD: Epoch 28 of 500. Elapsed time 39.11 min
FUNK_SVD: Processed 4228544 (100.0%) in 1.38 min. MSE loss 6.43E-03. Sample per second: 51196
FUNK_SVD: Epoch 29 of 500. Elapsed time 40.48 min
FUNK_SVD: Processed 4228544 (100.0%) in 1.38 min. MSE loss 3.92E-03. Sample per second: 51251
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 13642 (100.0%) in 8.40 sec. Users per second: 1624
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0002712, PRECISION_RECALL_MIN_DEN: 0.0003022, RECALL: 0.0000879, MAP: 0.0000756, MAP_MIN_DEN: 0.0000808, MRR: 0.0007560, NDCG: 0.0002768, F1: 0.0001327, HIT_RATE: 0.0027122, ARHR_ALL_H

FUNK_SVD: Epoch 54 of 500. Elapsed time 1.25 hour
FUNK_SVD: Processed 4228544 (100.0%) in 1.38 min. MSE loss 2.99E-03. Sample per second: 51218
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 13642 (100.0%) in 8.41 sec. Users per second: 1621
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0002419, PRECISION_RECALL_MIN_DEN: 0.0002419, RECALL: 0.0000399, MAP: 0.0000657, MAP_MIN_DEN: 0.0000657, MRR: 0.0006573, NDCG: 0.0002337, F1: 0.0000685, HIT_RATE: 0.0024190, ARHR_ALL_HITS: 0.0006573, NOVELTY: 0.0104278, AVERAGE_POPULARITY: 0.0012822, DIVERSITY_MEAN_INTER_LIST: 0.9841398, DIVERSITY_HERFINDAHL: 0.9984068, COVERAGE_ITEM: 0.2841796, COVERAGE_ITEM_CORRECT: 0.0016612, COVERAGE_USER: 0.9994139, COVERAGE_USER_CORRECT: 0.0024176, DIVERSITY_GINI: 0.0703011, SHANNON_ENTROPY: 10.5230289, RATIO_DIVERSITY_HERFINDAHL: 0.9987921, RATIO_DIVERSITY_GINI: 0.2835523, RATIO_SHANNON_ENTROPY: 0.8493075, RATIO_AVERAGE_POPULARITY: 0.0063180, RATIO_NOVELTY: 0.0490628, 

FUNK_SVD: Epoch 55 of 500. Elapsed time 1

FUNK_SVD: Processed 4228608 (100.0%) in 1.98 min. MSE loss 3.76E-04. Sample per second: 35646
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 13642 (100.0%) in 8.26 sec. Users per second: 1652
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0011362, PRECISION_RECALL_MIN_DEN: 0.0011362, RECALL: 0.0001250, MAP: 0.0003305, MAP_MIN_DEN: 0.0003305, MRR: 0.0031700, NDCG: 0.0011232, F1: 0.0002253, HIT_RATE: 0.0109955, ARHR_ALL_HITS: 0.0032374, NOVELTY: 0.0093173, AVERAGE_POPULARITY: 0.0064862, DIVERSITY_MEAN_INTER_LIST: 0.9989193, DIVERSITY_HERFINDAHL: 0.9998846, COVERAGE_ITEM: 0.8200897, COVERAGE_ITEM_CORRECT: 0.0084169, COVERAGE_USER: 0.9994139, COVERAGE_USER_CORRECT: 0.0109890, DIVERSITY_GINI: 0.4699226, SHANNON_ENTROPY: 13.4169390, RATIO_DIVERSITY_HERFINDAHL: 1.0002705, RATIO_DIVERSITY_GINI: 1.8953859, RATIO_SHANNON_ENTROPY: 1.0828733, RATIO_AVERAGE_POPULARITY: 0.0319597, RATIO_NOVELTY: 0.0438382, 

FUNK_SVD: Epoch 15 of 500. Elapsed time 29.90 min
FUNK_SVD: Processed 4228608 (100.0%) in 1

FUNK_SVD: Processed 4228544 (100.0%) in 1.70 min. MSE loss 1.02E-02. Sample per second: 41428
FUNK_SVD: Epoch 6 of 500. Elapsed time 10.29 min
FUNK_SVD: Processed 4228544 (100.0%) in 1.69 min. MSE loss 9.45E-03. Sample per second: 41625
FUNK_SVD: Epoch 7 of 500. Elapsed time 11.98 min
FUNK_SVD: Processed 4228544 (100.0%) in 1.70 min. MSE loss 8.78E-03. Sample per second: 41465
FUNK_SVD: Epoch 8 of 500. Elapsed time 13.67 min
FUNK_SVD: Processed 4228544 (100.0%) in 1.71 min. MSE loss 8.20E-03. Sample per second: 41269
FUNK_SVD: Epoch 9 of 500. Elapsed time 15.36 min
FUNK_SVD: Processed 4228544 (100.0%) in 1.70 min. MSE loss 7.68E-03. Sample per second: 41492
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 13642 (100.0%) in 8.25 sec. Users per second: 1653
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0039071, PRECISION_RECALL_MIN_DEN: 0.0039079, RECALL: 0.0004642, MAP: 0.0011244, MAP_MIN_DEN: 0.0011246, MRR: 0.0108141, NDCG: 0.0038386, F1: 0.0008298, HIT_RATE: 0.0376778, ARHR_ALL_HITS:

...

FUNK_SVD: Epoch 18 of 500. Elapsed time 19.79 min
FUNK_SVD: Processed 4228608 (100.0%) in 1.09 min. MSE loss 3.64E-03. Sample per second: 64814
FUNK_SVD: Epoch 19 of 500. Elapsed time 20.86 min
FUNK_SVD: Processed 4228608 (100.0%) in 1.08 min. MSE loss 1.11E-03. Sample per second: 65388
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 13642 (100.0%) in 9.00 sec. Users per second: 1515
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0001833, PRECISION_RECALL_MIN_DEN: 0.0001914, RECALL: 0.0000510, MAP: 0.0000508, MAP_MIN_DEN: 0.0000545, MRR: 0.0005079, NDCG: 0.0001843, F1: 0.0000798, HIT_RATE: 0.0018326, ARHR_ALL_HITS: 0.0005079, NOVELTY: 0.0110556, AVERAGE_POPULARITY: 0.0005732, DIVERSITY_MEAN_INTER_LIST: 0.3311011, DIVERSITY_HERFINDAHL: 0.9331077, COVERAGE_ITEM: 0.0648430, COVERAGE_ITEM_CORRECT: 0.0007752, COVERAGE_USER: 0.9994139, COVERAGE_USER_CORRECT: 0.0018315, DIVERSITY_GINI: 0.0018573, SHANNON_ENTROPY: 4.5324013, RATIO_DIVERSITY_HERFINDAHL: 0.9334678, RATIO_DIVERSITY_GINI: 0.007491

FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0957704, PRECISION_RECALL_MIN_DEN: 0.0959617, RECALL: 0.0143749, MAP: 0.0411409, MAP_MIN_DEN: 0.0412064, MRR: 0.2560055, NDCG: 0.1034890, F1: 0.0249977, HIT_RATE: 0.5909691, ARHR_ALL_HITS: 0.3238646, NOVELTY: 0.0062605, AVERAGE_POPULARITY: 0.3371258, DIVERSITY_MEAN_INTER_LIST: 0.9787332, DIVERSITY_HERFINDAHL: 0.9978661, COVERAGE_ITEM: 0.3999668, COVERAGE_ITEM_CORRECT: 0.0469572, COVERAGE_USER: 0.9994139, COVERAGE_USER_CORRECT: 0.5906227, DIVERSITY_GINI: 0.0686470, SHANNON_ENTROPY: 10.2143834, RATIO_DIVERSITY_HERFINDAHL: 0.9982513, RATIO_DIVERSITY_GINI: 0.2768806, RATIO_SHANNON_ENTROPY: 0.8243969, RATIO_AVERAGE_POPULARITY: 1.6611330, RATIO_NOVELTY: 0.0294558, 

FUNK_SVD: New best model found! Updating.
FUNK_SVD: Epoch 10 of 500. Elapsed time 3.89 min
FUNK_SVD: Processed 4228544 (100.0%) in 21.59 sec. MSE loss 1.24E-02. Sample per second: 195844
FUNK_SVD: Epoch 11 of 500. Elapsed time 4.25 min
FUNK_SVD: Processed 4228544 (100.0%) in 22.15 sec. MSE loss

EvaluatorHoldout: Processed 13642 (100.0%) in 9.44 sec. Users per second: 1446
SearchBayesianSkopt: New best config found. Config 18: {'epochs': 10, 'num_factors': 35, 'sgd_mode': 'adagrad', 'batch_size': 32, 'item_reg': 0.00017366160017684616, 'user_reg': 0.00036964278347072086, 'learning_rate': 0.0005217220866779753} - results: PRECISION: 0.0957704, PRECISION_RECALL_MIN_DEN: 0.0959617, RECALL: 0.0143749, MAP: 0.0411409, MAP_MIN_DEN: 0.0412064, MRR: 0.2560055, NDCG: 0.1034890, F1: 0.0249977, HIT_RATE: 0.5909691, ARHR_ALL_HITS: 0.3238646, NOVELTY: 0.0062605, AVERAGE_POPULARITY: 0.3371258, DIVERSITY_MEAN_INTER_LIST: 0.9787332, DIVERSITY_HERFINDAHL: 0.9978661, COVERAGE_ITEM: 0.3999668, COVERAGE_ITEM_CORRECT: 0.0469572, COVERAGE_USER: 0.9994139, COVERAGE_USER_CORRECT: 0.5906227, DIVERSITY_GINI: 0.0686470, SHANNON_ENTROPY: 10.2143834, RATIO_DIVERSITY_HERFINDAHL: 0.9982513, RATIO_DIVERSITY_GINI: 0.2768806, RATIO_SHANNON_ENTROPY: 0.8243969, RATIO_AVERAGE_POPULARITY: 1.6611330, RATIO_NOVELTY:

FUNK_SVD: Processed 4228544 (100.0%) in 26.16 sec. MSE loss 1.62E-02. Sample per second: 161616
FUNK_SVD: Epoch 22 of 500. Elapsed time 10.13 min
FUNK_SVD: Processed 4228544 (100.0%) in 25.80 sec. MSE loss 1.62E-02. Sample per second: 163927
FUNK_SVD: Epoch 23 of 500. Elapsed time 10.56 min
FUNK_SVD: Processed 4228544 (100.0%) in 26.42 sec. MSE loss 1.62E-02. Sample per second: 160034
FUNK_SVD: Epoch 24 of 500. Elapsed time 10.98 min
FUNK_SVD: Processed 4228544 (100.0%) in 25.98 sec. MSE loss 1.61E-02. Sample per second: 162740
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 13642 (100.0%) in 9.11 sec. Users per second: 1498
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0039510, PRECISION_RECALL_MIN_DEN: 0.0039510, RECALL: 0.0004920, MAP: 0.0011424, MAP_MIN_DEN: 0.0011424, MRR: 0.0110085, NDCG: 0.0038848, F1: 0.0008750, HIT_RATE: 0.0381176, ARHR_ALL_HITS: 0.0112164, NOVELTY: 0.0089233, AVERAGE_POPULARITY: 0.0212497, DIVERSITY_MEAN_INTER_LIST: 0.9992191, DIVERSITY_HERFINDAHL: 0.9999146

FUNK_SVD: Epoch 14 of 500. Elapsed time 24.51 min
FUNK_SVD: Processed 4228608 (100.0%) in 1.74 min. MSE loss 6.39E-08. Sample per second: 40575
FUNK_SVD: Validation begins...
EvaluatorHoldout: Processed 13642 (100.0%) in 9.53 sec. Users per second: 1431
FUNK_SVD: CUTOFF: 10 - PRECISION: 0.0006451, PRECISION_RECALL_MIN_DEN: 0.0006459, RECALL: 0.0001009, MAP: 0.0001623, MAP_MIN_DEN: 0.0001625, MRR: 0.0015934, NDCG: 0.0005979, F1: 0.0001745, HIT_RATE: 0.0063041, ARHR_ALL_HITS: 0.0016081, NOVELTY: 0.0096037, AVERAGE_POPULARITY: 0.0038642, DIVERSITY_MEAN_INTER_LIST: 0.8285040, DIVERSITY_HERFINDAHL: 0.9828443, COVERAGE_ITEM: 0.4890636, COVERAGE_ITEM_CORRECT: 0.0042638, COVERAGE_USER: 0.9994139, COVERAGE_USER_CORRECT: 0.0063004, DIVERSITY_GINI: 0.0954486, SHANNON_ENTROPY: 9.0864217, RATIO_DIVERSITY_HERFINDAHL: 0.9832237, RATIO_DIVERSITY_GINI: 0.3849823, RATIO_SHANNON_ENTROPY: 0.7333598, RATIO_AVERAGE_POPULARITY: 0.0190401, RATIO_NOVELTY: 0.0451854, 

FUNK_SVD: Epoch 15 of 500. Elapsed time 26